In [1]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import Preprocessing as pp

In [2]:
temp_data_folder = "../Data/Temp_Data/"
image_path = "../Images/"
image_subfolder = "Proposal/"
image_folder = image_path + image_subfolder

pp.checkFolder(temp_data_folder)
pp.checkFolder(image_path)
pp.checkFolder(image_folder)

Folder already exists.
Folder already exists.
Folder already exists.


In [3]:
df_1 = pp.openCSVFile('experiment_1_short.csv', temp_data_folder)
df_4b = pp.openCSVFile('experiment_4b_short.csv', temp_data_folder)
df_9 = pp.openCSVFile('experiment_9_short.csv', temp_data_folder)

In [4]:
dataset = df_4b[['time', 'input_voltage', 'el_power']]

In [5]:
numpy.random.seed(7)

In [6]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

TypeError: '(slice(0, 7061, None), slice(None, None, None))' is an invalid key

In [ ]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [75]:
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [76]:
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [77]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=2, batch_size=1, verbose=2)

Epoch 1/2
7057/7057 - 7s - loss: 0.0027
Epoch 2/2
7057/7057 - 6s - loss: 1.2612e-05


In [78]:
df_4b.columns

Index(['time', 'input_voltage', 'el_power'], dtype='object')

In [105]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
print(trainX[:,0])
print(trainPredict)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
# trainY = scaler.inverse_transform([trainY])
# testPredict = scaler.inverse_transform(testPredict)
# testY = scaler.inverse_transform([testY])
# calculate root mean squared error
# trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
# print('Train Score: %.2f RMSE' % (trainScore))
# testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
# print('Test Score: %.2f RMSE' % (testScore))

[[0.00000000e+00 9.48856628e-05 1.89771326e-04]
 [9.48856628e-05 1.89771326e-04 2.84656988e-04]
 [1.89771326e-04 2.84656988e-04 3.79542651e-04]
 ...
 [6.69323465e-01 6.69418351e-01 6.69513237e-01]
 [6.69418351e-01 6.69513237e-01 6.69608122e-01]
 [6.69513237e-01 6.69608122e-01 6.69703008e-01]]
[[0.0086686 ]
 [0.00875212]
 [0.00883566]
 ...
 [0.6666476 ]
 [0.66673386]
 [0.66681993]]


ValueError: non-broadcastable output operand with shape (7057,1) doesn't match the broadcast shape (7057,3)

In [87]:
plt.plot(dataset['time'], testPredict)
plt.plot(dataset['time'], dataset['el_power'])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()